In [3]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [4]:

# Fetch the JSON data from the URL
url = "https://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_T34-E-2"
response = requests.get(url)
json_data = response.json()

# Flatten the JSON data
#print(pd.json_normalize(json_data))
df = pd.json_normalize(json_data, "Grunntyper",["Kategori","Navn", ["Kode","Id"], "Målestokk"] ,record_prefix="Gr")
print(df.size)
print(df.columns)
# Display the DataFrame
print(df)

32
Index(['GrNavn', 'GrKategori', 'GrKode.Id', 'GrKode.Definisjon', 'Kategori',
       'Navn', 'Kode.Id', 'Målestokk'],
      dtype='object')
                                                         GrNavn GrKategori  \
0                                         intermediær bakli-hei  Grunntype   
1                           intermediær tørkeutsatt kystlynghei  Grunntype   
2                                intermediær normal kystlynghei  Grunntype   
3  intermediær normal kystlynghei på veksefuktig og fuktig mark  Grunntype   

   GrKode.Id  \
0   NA T34-4   
1   NA T34-6   
2   NA T34-5   
3  NA T34-12   

                                                            GrKode.Definisjon  \
0   http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_T34-4   
1   http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_T34-6   
2   http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_T34-5   
3  http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hen

In [5]:
import pandas as pd
import requests
import json

# Function to flatten nested JSON
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out


In [6]:
url2 = 'https://nin-kode-api.artsdatabanken.no/v2.3/koder/allekoder'
response = requests.get(url2)# tar ca. 49 sek
json_data = response.json()
allekoder = pd.json_normalize(json_data)

In [7]:
#url2 = allekoder[allekoder['Kategori']=="Grunntype"]['Kode.Definisjon'][1027]
res = pd.DataFrame()
#idx = str(allekoder[allekoder['Kategori']=="Grunntype"]['Kode.Definisjon']).split()
idx = allekoder[allekoder['Kategori']=="Grunntype"]['Kode.Definisjon']

for i in idx:
    print(i)
    #url2 = allekoder[allekoder['Kategori']=="Grunntype"]['Kode.Definisjon'][i]
    response = requests.get(i)# tar ca. 49 sek
    json_data_sub = response.json()
    subdf = pd.json_normalize(json_data_sub)
    # Flatten the JSON data
    flattened_data = flatten_json(json_data_sub)
    # Convert to DataFrame
    flattened_data = pd.DataFrame([flattened_data])
    res = pd.concat([res, flattened_data], axis=0, ignore_index=True, sort=False)

print(res)

http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H1-5
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H1-4
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H1-3
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H1-2
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H1-1
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-8
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-7
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-6
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-5
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-4
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-3
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-2
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/koder/hentkode/NA_H2-1
http://adb-prod-ninkapi-as.azurewebsites.net/v2.3/k

In [8]:
d2 = res.filter(regex='Navn|Kategori|Id')
d3 = d2.filter(regex="Kart")
d3_b = d2.filter(regex="Basis")
d4 = pd.concat([res.filter(regex='Navn|Kategori'), d3_b], axis=1)
print(d4)


                                                                                                                          Navn  \
0                                                                                                  epipelagiale kystvannmasser   
1                                                                                                abyssopelagiale havvannmasser   
2                                                                                                 bathypelagiale havvannmasser   
3                                                                                                  mesopelagiale havvannmasser   
4                                                                                                   epipelagiale havvannmasser   
5                                                                                      middels stor eller liten brakkvannspoll   
6                                                                                         